# Searching EarthCODE with the OSC library

This notebook shows how to use the EarthCODE OSC search function.

You can directly use natural language for your search and write various queries

In [8]:
from earthcode.search import search

search("forest fires")[0]


<Collection id=seasfire-cube>

Search within a known collection id if you already know what you want. This keeps results tight and avoids scanning the whole catalog.


In [9]:
search(collection_ids="seasfire-cube")


[<Collection id=seasfire-cube>]

Use `bbox=[minx, miny, maxx, maxy]` to limit by geography. By default the bbox only needs to intersect the product footprint. Example: snow data over the Alps.


In [12]:
alps_bbox = [5.95591129, 45.81799493, 10.49229402, 47.80846475]
search("snow data", limit=1, bbox=alps_bbox)[0]


<Collection id=binary-wet-snow-s14science-snow>

Switch `intersects=False` when you need the whole footprint to sit inside the bbox (containment instead of overlap).


In [13]:
search("snow data", limit=1, bbox=alps_bbox, intersects=False)[0]


<Collection id=snow-cover-alpglacier>

You can also search for variables to find relevant parameters or measured quantities.


In [14]:
search("chlorophyll", type="variables", limit=2)[0]


<Catalog id=chlorophyll-concentration>

And search for EO missions to see which platforms match your query.


In [15]:
search("sentinel", type="eo-missions", limit=2)[0]


<Catalog id=sentinel-5p>